In [19]:
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import urllib
import requests
import pandas as pd
import re
import time

In [2]:
new_data = pd.read_csv("./Lianjia_info_partial.csv")
new_data.drop(new_data.columns[0], axis=1, inplace=True)
new_data = new_data.reset_index(drop = True)
new_data.head()

,高德ID,链家ID,高德名称,链家名称
0,B0FFF3BZ0R,5011102206884,苏宁荣悦怡庭,苏宁荣悦
1,B00155L48N,5011000019384,庙泾新村,庙泾新村
2,B00156ZH9S,5011102206699,绿地未来中心,绿地未来中心
3,B0FFF75YUY,5011063715958,馨雅名筑,馨雅名筑
4,B0FFG1OG38,5011000019801,半岛君望,半岛君望
...,...,...,...,...
101,B00155Q8SM,5011000004160,同盛南桥花园,同盛南桥花园
102,B0FFKBCKM8,5020036484143511,光语著,光语著
103,B00155LPOQ,5011000003475,九华新园,九华新园
104,B00155N813,5011000011571,四季花城,百合苑(公寓)


In [3]:
# Select a random agent IP adress, and dim the headers
proxy = {'https': '102.164.212.54:8080'}
headers = {'User-Agent':
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36 Edg/84.0.522.61"}

In [26]:
# 按照链家网的小区ID编号搜索小区详细信息
resid_table = pd.DataFrame()
for resid_id in new_data['链家ID']:         
    # 构造小区网页地址
    url_resid = 'https://sh.lianjia.com/xiaoqu/{}'.format(resid_id)
    # 访问
    response = requests.get(url_resid, headers = headers)
    # soup一下美美的
    soup = bs(response.text, 'html.parser')
    # 信息截取
    # 网页源码中 label_list 和 content_list 两个关键内容
    label_list = soup.find_all(class_ = 'xiaoquInfoLabel')
    content_list = soup.find_all(class_ = 'xiaoquInfoContent')
    # 将抓到的信息存进DataFrame
    info_dict = defaultdict(list) #构造一个空字典且将其实例化的方法，便于append
    info_dict['链家ID'].append(resid_id)
    for i,j in zip(label_list,content_list):
        info_dict[i.get_text()].append(j.get_text())
    resid_table = resid_table.append(pd.DataFrame(info_dict),ignore_index=True)
    time.sleep(1)

In [27]:
resid_table.head()

,链家ID,建筑年代,建筑类型,物业费用,物业公司,开发商,楼栋总数,房屋总数,附近门店
0,5011102206884,2015年建成,板楼,2.5元/平米/月,江苏银河物业管理有限公司上海分公司,上海奉贤苏宁电器有限公司,12栋,592户,暂无门店信息
1,5011000019384,2006年建成,板楼,0.7元/平米/月,金维邦物业,上海华馨置业有限公司,44栋,454户,暂无门店信息
2,5011102206699,2008年建成,塔楼/板楼,2元/平米/月,暂无信息,绿地集团,8栋,2623户,暂无门店信息
3,5011063715958,2015年建成,板楼,1.96元/平米/月,上海永佳物业管理有限公司,上海卓翰房地产开发有限公司,14栋,686户,正阳世纪星城店/上海市奉贤区南奉公路8889弄77号2层、79号1层及2层
4,5011000019801,2013年建成,板楼,6.62元/平米/月,上海紫泰物业管理有限公司,上海绿庭四季花城房地产开发有限公司,81栋,81户,奉浦一店/上海市奉贤区环城东路515号101室
...,...,...,...,...,...,...,...,...,...
101,5011000004160,2011年建成,板楼,1.92至5元/平米/月,中房物业,同盛南桥置业有限公司,13栋,664户,南桥新苑一店/上海市奉贤区环城东路883弄13号1-2层
102,5020036484143511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,5011000003475,2006年建成,板楼,0.8元/平米/月,上海营巢物业管理有限公司,上海营巢房产开发有限公司,156栋,838户,南桥新苑一店/上海市奉贤区环城东路883弄13号1-2层
104,5011000011571,2006年建成,板楼,1.51元/平米/月,大至物业,上海绿庭集团有限公司,29栋,702户,奉浦一店/上海市奉贤区环城东路515号101室


In [29]:
resid_table.to_csv('./Lianjia_info_all.csv',encoding='utf_8_sig')